In [1]:
#import necessary libraries
import ast
import pandas as pd
import time
import json
import numpy as np

#import necessary modules to work in R within python
import rpy2.robjects as r

#import pandas2ri for pandas to R dataframe conversion
from rpy2.robjects import pandas2ri
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
pandas2ri.activate()

#this allows me to import libraries in python/R, similar to library('lavaan')
from rpy2.robjects.packages import importr
utils = importr("lavaan")
utils = importr("eRm")

#read in the all-outcomes text file
outcomes1_txt = open('outcomes.txt')
outcomes1_str = outcomes1_txt.read()
outcomes1_json = json.loads(outcomes1_str)
outcomes1 = pd.DataFrame(outcomes1_json)

#read in the assessment data
json_file = open('assessment_data.json')
json_str = json_file.read()
json_data = json.loads(json_str)

#import the lti_context_id to root_guid_id report
guid_to_context = pd.read_csv('lti_context_to_outcome_guid.csv',encoding='utf-8')
guid_to_context = guid_to_context[['RootOutcomeGuid','LtiContextId']]   #I only need these two columns so I get rid of the rest
guid_to_context.index = guid_to_context.LtiContextId   #assign LtiContextId as the index (to easily match values)

#flatten assessment data down into one level (at the question level)
quiz_data = []
for submission in json_data:
    for question in submission['question_responses']:
        quiz_data.append({
                            'question_id':question['ident'],
                            'score':question['score'],
                            'outcome_guid':question['outcome_guid'],
                            'quiz_id':submission['data']['quiz_id'],
                            'quiz_type':submission['quiz_type'],
                            'lti_user_id':submission['lti_user_id'],
                            'attempt':submission['data']['attempt'],
                            'primary_outcome_guid':submission['primary_outcome_guid'],
                            'created_at':submission['created_at'],
                            'tc_lti_guid':submission['tc_lti_guid'],
                            'lti_context_id':submission['lti_context_id'],
                            'enrollment_id':submission['enrollment_id']  
                         })

#create dataframe from all quiz data
df = pd.DataFrame(quiz_data)
summative = df[df.quiz_type == 'summative']   #only select summative questions
summative = summative[summative.attempt == 0]   #only keep first attempt for each student

#create new column that is the outcome and question_id combined, separated by a ":"
summative['outquest_map'] = summative.outcome_guid.astype(str) + ':' + summative.question_id.astype(str)

#create new dataframe that has each unique outcome:question_id along with a number from 1 to the length of the outcome:question_id list
outquest_to_id_default = pd.DataFrame(pd.Series(range(len(summative.outquest_map.unique()))).astype(str).values,index=summative.outquest_map.unique(),columns=['id'])
summative['outquest'] = summative['outquest_map'].map(lambda x: outquest_to_id_default.loc[x,'id'])   #assigns a new id to each question attempt record
summative = summative.drop_duplicates()   #drops duplicates in case there are duplicate values

#function to get the RootOutcomeGuid from the LtiContextId to Root_Guid file given an LtiContextId
#this is trickier than it should have been because there were float values, int values, str values, and series values
#this was because some LtiContextIds had multiple RootOutcomeGuids (low enrollment courses)
def get_root_guid(cell):
    if cell != cell:   #handles None or missing values
        return None
    else:
        if str(cell) in guid_to_context.index:   #checks if the LtiContextId is located in the mapping file
            result = guid_to_context.loc[str(cell),'RootOutcomeGuid']   #gets the RootOutcomeGuid
            if type(result) == float:   #handles float values
                result = str(result)
            if len(result) == 2:   #handles list values
                try:
                    cell_guid = result[0]
                except:
                    return None
            else:                
                cell_guid = str(result)
        else:
            return None
    
    return cell_guid

#run the get_root_guid function to get RootOutcomeGuid from Lti_Context_Id
summative['root_guid'] = summative['lti_context_id'].map(get_root_guid)

#courses in the outcomes file
courses_df = outcomes1[outcomes1.root_guid != outcomes1.root_guid]

#loop through each course in the assessment data (based on the outcomes without a root_guid, which are the course root_guids)
for root_guid_course in range(len(courses_df.guid.values)):
    
    #extract root_guid and course name
    root_guid = courses_df.guid.values[root_guid_course]
    course_name = courses_df.short_title.values[root_guid_course]
    
    #creates an outcome to parent outcome to question mapping dataframe
    question_df = summative[summative.root_guid == root_guid][['outcome_guid','primary_outcome_guid','outquest']]
    question_df = question_df.drop_duplicates()
    
    #only keeps outcomes that have the root_guid corresponding to the course
    outcomes2 = outcomes1[outcomes1.root_guid == root_guid]
    outcomes2 = outcomes2[['guid','parent_guid','short_title','title']]
    
    #creates a question by student matrix I can use for CFA
    #only keeps assessment data with the course root_guid
    #questions are columns, rows are students
    pivot_table = summative[summative.root_guid == root_guid].pivot_table(index='lti_user_id', columns='outquest', values='score')
    
    #create a list of students that have at least 5 summative assessments in this specific course
    greater_than_five_check = summative[['lti_user_id','quiz_id']].drop_duplicates().groupby('lti_user_id').count()
    students = list(greater_than_five_check[greater_than_five_check.quiz_id > 5].index)

    #filter out students with low assessment numbers
    pivot_table = pivot_table[pivot_table.index.isin(students)]

    #initiate lists to save final analysis objects
    converge_outcomes = []
    no_converge_outcomes = []
    less_than_3_questions = []
    final_IRT = []
    dropped_IRT_questions = []
    dropped_CFA_questions = []
    normal_rasch_model = []

    #number of question responses required
    response_threshold = 100

    #loop through each enabling outcome for each parent_outcome and run a CFA and IRT model
    #we are conducting the analysis at the enabling outcome level
    count = 0
    for parent_outcome in outcomes2.parent_guid.unique():
        outcomes = outcomes2[outcomes2.parent_guid == parent_outcome].guid.unique()
        
        for outcome in outcomes:    
            #uses a unique outcome + questionID identifier because neither outcome nor question is unique by itself
            questions2 = question_df[question_df.outcome_guid == outcome].outquest.unique()

            #slice pivot_table data based on questions pertaining to outcome
            CFA_data = pivot_table[questions2] 
            
            #if there is no data for these questions, skip the outcome
            if len(CFA_data) == 0:
                count += 1
                continue
            
            #I don't really know why I drop duplicates here, but it shouldn't hurt anything
            CFA_data = CFA_data.T.drop_duplicates().T
            
            #the thresh parameter means I can make sure each person in the dataset has at least one response to one question
            CFA_data = CFA_data.dropna(axis=0,thresh=1)
            
            #these are the questions left after I drop all students with missing values on all problems
            #questions should essentially be the same as questions2
            questions = [x for x in questions2 if x in CFA_data.columns]

            #remove questions with less than 100 responses
            question_count = CFA_data.count()
            greater_than_100_questions = question_count[question_count.values > response_threshold].index
            CFA_data = CFA_data[greater_than_100_questions]

            #questions that were removed for too few responses
            removed_questions = list(question_count[question_count.values <= 100].index)
            dropped_CFA_questions.append(removed_questions)
            dropped_CFA_questions.append([x for x in questions2 if x not in CFA_data.columns])
            
            #IRT model to identify difficulty for each item
            IRT_data = CFA_data.copy()

            #throw out questions without 0's or 1's because it messes up the algorithm
            #I have to iterate through the dataset 5 times because if I drop students (didn't respond to two items only one)
            #then questions might not have a 1 or 0 anymore. And if I drop questions, students might not have more than two
            #responses on questions. So it's basically a brute force optimization algorithm to get correct data
            for x in range(5):
                IRT_data = IRT_data[IRT_data.columns[(IRT_data.min() == 0).values&(IRT_data.max() == 1).values]]
                IRT_data = IRT_data.dropna(axis=0,thresh=2)
                if len(IRT_data) == 0: #we don't have any IRT data. Break out of the for loop.
                    continue
                IRT_data = IRT_data.T.drop_duplicates().T

            if len(IRT_data) == 0:   #we don't have any IRT data, so skip running everything
                continue
            
            #convert IRT_data into an R object
            r_IRT_data = pandas2ri.py2ri(IRT_data)

            #dropped IRT questions
            dropped_IRT_question = [x for x in CFA_data.columns if x not in IRT_data.columns]
            if len(IRT_data.columns) < 3:
                for x in IRT_data:
                    dropped_IRT_question.append(x)
                
                continue #stop if there aren't enough questions to run a model

            #running the model
            try:
                r.r('''res = LPCM({}, groupvec = 1, se = TRUE, sum0 = TRUE)'''.format(r_IRT_data.r_repr()))
            except:
                #if it doesn't work, we probably need a normal rasch model instead of a partial credit model
                normal_rasch_model.append({
                        'outcome':outcome,
                        'parent_outcome':parent_outcome
                        })

#                 #for now I am stopping the script. Eventually I will have it run a rasch model, if needed.
#                 for xyz in IRT_data.columns:
#                     dropped_IRT_questions.append(xyz)
#                 continue
#                 for x in IRT_data.columns:
#                     IRT_data = IRT_data[~((IRT_data[x] > 0)&(IRT_data[x] < 1))]
#                 IRT_data = IRT_data.dropna(axis=0,thresh=2)
#                 r_IRT_data = pandas2ri.py2ri(IRT_data)
#                 r.r('''res = RM(as.matrix({}), se = TRUE, sum0 = TRUE)'''.format(r_IRT_data.r_repr()))
#                 print "Abnormal"

            #get means, min, and max for dropped questions for diagnostic purposes
            dropped_means = [CFA_data[x].mean() for x in dropped_IRT_question]
            dropped_min = [CFA_data[x].min() for x in dropped_IRT_question]
            dropped_max = [CFA_data[x].max() for x in dropped_IRT_question]

            #save the mean score of the dropped questions
            dropped_IRT_questions.append({
                    'dropped_IRT_question':dropped_IRT_question,
                    'dropped_means':dropped_means,
                    'dropped_min':dropped_min,
                    'dropped_max':dropped_max,
                    'outcome':outcome
                })

            #extracting coefficients from the model
            #question_difficulty is actually question_easiness
            question_difficulty = list(r.r('''res['betapar']''')[0])
            etapar = list(r.r('''res['etapar']''')[0])
            questionsIRT = list(IRT_data.columns)   #questions used in the IRT model

            #final results
            IRT_results = {'qIRT':questionsIRT,
                           'questions':questions,
                           'question_difficulty':question_difficulty,
                           'dropped_IRT_questions':dropped_IRT_questions,
                           'etapar':etapar,
                           'outcome':outcome}
            final_IRT.append(IRT_results)

            #if we have less than 4 items per outcome we have to run a special CFA model (with constraints in order for it to run)
            #we are simply saving these outcomes right now
            if len(CFA_data.columns) < 4:
                less_than_3_questions.append({
                        'outcome':outcome,
                        'parent_outcome':parent_outcome,
                        'number_of_questions':len(CFA_data.columns)
                    })
                count += 1
                continue #stop because we can't run a CFA

            #R adds X's before these variables when the dataframe is transferred from pandas to R.
            IVs = 'X' + ' + X'.join(CFA_data.columns)
            DV = '{} =~ '.format('outcome')
            
            #create a string formula to use in the CFA
            formula = str(DV + IVs)

            #creates an R dataframe out of a pandas dataframe
            r_CFA_data = pandas2ri.py2ri(CFA_data)

            #runs a CFA using the lavaan package
            #right now I am catching all errors and just stopping if it doesn't work
            try:
                r.r('''test <- cfa('{}', data={}, missing = "ML",std.lv=TRUE)'''.format(formula, r_CFA_data.r_repr()))
                r.r('''summary(test,standardized=TRUE)''')

                #extract all fit statistics and parameter estimates from the model
                aic = r.r('''AIC(test)''')[0]
                bic = r.r('''BIC(test)''')[0]
                rmsea = r.r('''fitMeasures(test, "rmsea")''')[0]
                cfi = r.r('''fitMeasures(test, "cfi")''')[0]
                tli = r.r('''fitMeasures(test, "tli")''')[0]
                srmr = r.r('''fitMeasures(test, "srmr")''')[0]
                parameter_estimates = r.r('''parameterEstimates(test)''')
                mod_indices = r.r('''modindices(test)''')
                converge_outcomes.append({
                        'aic':aic,
                        'bic':bic,
                        'rmsea':rmsea,
                        'cfi':cfi,
                        'tli':tli,
                        'srmr':srmr,
                        'parameter_estimates':parameter_estimates,
                        'mod_indices':mod_indices,
                        'CFA_data':CFA_data.count(),
                        'outcome':outcome,
                        'parent_outcome':parent_outcome
                    })
                count += 1
            except:
                #there was some error in running the model
                count += 1
                warnings = r.r('''warnings()''')
                no_converge_outcomes.append({
                        'warnings':warnings,
                        'CFA_data':CFA_data.count(),
                        'outcome':outcome,
                        'parent_outcome':parent_outcome
                    })

    #creates a dataframe from CFA converged models
    converge_df = pd.DataFrame(converge_outcomes)

    if len(converge_df) == 0:   #if we don't have any data, skip returning results
        continue
    
    #this function, when vectorized, will take pandas series and assess model fit quality
    #based on four cutoff points (cfi,tli,rmsea,srmr)
    def assess_quality(cfi,tli,rmsea,srmr):
        count = 0
        if cfi > .9:
            count += 1
        if tli > .9:
            count += 1
        if rmsea < .08:
            count += 1
        if srmr < .08:
            count += 1
        return count

    #this will vectorize the function to be used on a pandas series
    assess_quality = np.vectorize(assess_quality)
    
    #run the assess_quality function to get a number for model fit
    converge_df['model_fit'] = assess_quality(converge_df.cfi,converge_df.tli,converge_df.rmsea,converge_df.srmr)

    #model fit summary for this course
    #Excellent model fit: 4 means all 4 statistics meet the rules of thumb (CFI>.9,TLI>.9,SRMR<.08,RMSEA<.08)
    #Good model fit: 3 means the majority still show good model fit
    #Questionable model fit: 2 means it could go either way, improvements needed
    #Poor model fit: 1 or 0 means there is no evidence to suggest the model is fitting well

    #lambda functions to get the number of questions and their IDs for each analysis
    converge_df['questionIDs'] = converge_df['CFA_data'].map(lambda x: list(x.index))
    converge_df['question_count'] = converge_df['CFA_data'].map(lambda x: len(list(x.index)))

    #rename guid to outcome so I can merge it with my final dataset
    outcomes2.rename(columns={'guid':'outcome'},inplace=True)

    #this is the final outcomes level data file
    final_outcome_level_data = converge_df.copy()

    #final outcome level data merged together with other data
    final_outcome_level_data = pd.merge(final_outcome_level_data,outcomes2)

    #we don't need all columns, only keep the columns you want
    final_outcome_level_data = final_outcome_level_data[['parent_outcome','outcome','short_title','cfi','tli','srmr','rmsea','aic','bic','model_fit','question_count','questionIDs']]
    
    ###outcome level data is completed, next is question level data###
    
    #getting question level data
    questions = []
    response_count = []
    parameter_estimates = []
    
    #weird list comprehensions to get questions and response_counts into lists
    #these should probably just be normal functions, although the current implementation works
    converge_df['CFA_data'].map(lambda x: [questions.append(y) for y in x.index])
    converge_df['CFA_data'].map(lambda x: [response_count.append(y) for y in x.values])
    converge_df['parameter_estimates'].map(lambda x: parameter_estimates.append(x[x['op'] == '=~'][['rhs','est','se','z','pvalue']]))

    #initialize a blank dataframe for use in the following section
    CFA_question_data = pd.DataFrame(0,columns=['rhs','est','se','z','pvalue'],index=range(0))
    for x in parameter_estimates:   #add parameter estimates to CFA_question_data
        CFA_question_data.append(x)

    CFA_question_data['rhs'] = CFA_question_data['rhs'].map(lambda x: x[1:])   #parsing the rhs value
    
    #renaming values to be more easily interpreted
    CFA_question_data.rename(columns={'rhs':'question',
                          'est':'factor_loading',
                          'se':'se_factor_loading',
                          'z':'z_factor_loading',
                          'pvalue':'p_factor_loading'},inplace=True)
    
    #reset the index so you can use it in the next analyses
    CFA_question_data.reset_index(drop=True,inplace=True)

    questions_IRT = []
    difficulty_IRT = []
    theta_scales = []
    
    #weird list comprehensions to get IRT data into a dataframe
    #there is definitely a better way
    irtdf = pd.DataFrame(final_IRT)
    irtdf.qIRT.map(lambda x: [questions_IRT.append(y) for y in x])
    irtdf.question_difficulty.map(lambda x: [difficulty_IRT.append(y) for y in x])
    irtdf.etapar.map(lambda x: [theta_scales.append(y) for y in x])

    #create IRT dataframe from questions and difficulty parameter
    IRT_question_data = pd.DataFrame(list(zip(questions_IRT,difficulty_IRT))).rename(columns={0:'question',1:'difficulty'})

    #create CFA dataframe
    CFA_question_data2 = pd.DataFrame(list(zip(questions,response_count))).rename(columns={0:'question',1:'response_count'})

    #merge IRT, CFA, and missing CFA data together into a question_level_data dataframe
    question_level_data = IRT_question_data.merge(CFA_question_data2,on='question',how='outer')
    question_level_data = question_level_data.merge(CFA_question_data,on='question',how='outer')
    
    ###Final question level data###
    final_question_level_data = question_level_data.sort_values('difficulty')

    ####Final Output Files####
    # final_question_level_data
    # final_outcome_level_data
    # no_converge_outcomes
    # dropped_IRT_questions
    # dropped_CFA_questions

    #code found here: http://stackoverflow.com/questions/26068021/iterate-over-rows-and-expand-pandas-dataframe
    #flatten a dataframe when you have a series of lists for each item in the list in the series
    dfIn = final_outcome_level_data.copy()
    dfIn.loc[:, 'questionIDs'] = dfIn.questionIDs.apply(np.atleast_1d)
    all_locations = np.hstack(dfIn.questionIDs)
    all_names = np.hstack([[n]*len(l) for n, l in dfIn[['outcome','questionIDs']].values])
    dfOut = pd.DataFrame({'question':all_locations, 'outcome':all_names})

    #output is the final merge of all question/outcome datafiles
    output = dfOut.merge(final_outcome_level_data,on='outcome',how='outer').merge(final_question_level_data,on='question',how='outer')
    
    #add min, max, and mean to dropped IRT questions for diagnostic purposes
    for outcome in dropped_IRT_questions:
        for question in range(len(outcome['dropped_IRT_question'])):
            questionID = outcome['dropped_IRT_question'][question]
            questionmax = outcome['dropped_max'][question]
            questionmin = outcome['dropped_min'][question]
            questionmean = outcome['dropped_means'][question]
            output.loc[output.question == questionID,'question_min'] = questionmin
            output.loc[output.question == questionID,'question_max'] = questionmax

    #add in the actual questionID since we replaced it with the question/outcome combo ID
    outquest_to_id = outquest_to_id_default.copy()
    outquest_to_id['outquest'] = outquest_to_id.index
    outquest_to_id = outquest_to_id.set_index('id')
    outquest_to_id['questionID'] = outquest_to_id.outquest.map(lambda x: x.split(':')[1])
    output['questionID'] = output.question.map(lambda x: outquest_to_id.loc[x,'questionID'])

    #merge with mean_correct dataset to get mean correct values
    mean_correct_df = summative.groupby('question_id').mean()
    mean_correct_df['questionID'] = mean_correct_df.index
    mean_correct_df = mean_correct_df[['questionID','score']]
    output = output.merge(mean_correct_df,on='questionID')
    
    #merge with attempts dataset to get attempts for each question
    attempts_df = summative.groupby('question_id').count()
    attempts_df['questionID'] = attempts_df.index
    attempts_df = attempts_df[['questionID','score']]
    output = output.merge(attempts_df,on='questionID')
    
    
    #output the data to the root directory
    output.to_csv('Output//{}_{}_final_IRT-CFA_data.csv'.format(course_name,root_guid[:4]),index=False,encoding='utf-8')

    #adds outcomes to all questions (even if they didn't converge for CFA)
    outquest_to_id['outcome'] = outquest_to_id.outquest.map(lambda x: x.split(':')[0])
    output['outcome'] = output.question.map(lambda x: outquest_to_id.loc[x,'outcome'])

    #prepare dropped CFA questions for export
    CFA_questions = []
    for question_list in dropped_CFA_questions:
        for question in question_list:
            CFA_questions.append(question)

    final_CFA_list = []
    for question in CFA_questions:
        if question not in output.question.values:
            final_CFA_list.append(question)

    #prepare dropped IRT questions for export
    IRT_questions = []
    for question_list in dropped_IRT_questions:
        for question in question_list['dropped_IRT_question']:
            IRT_questions.append(question)

    final_IRT_list = []
    for question in IRT_questions:
        if question not in output.question.values:
            final_IRT_list.append(question)

    #export data to csv files named by course name and the first 4 characters of the root guid
    pivot_table[final_CFA_list].describe().T.to_csv('Output//{}_{}_dropped_CFA_questions.csv'.format(course_name,root_guid[:4]))
    if len(final_IRT_list) < 1:
        pass
    else:
        pivot_table[final_IRT_list].describe().T.to_csv('Output//{}_{}_dropped_IRT_questions.csv'.format(course_name,root_guid[:4]))
    pd.DataFrame(no_converge_outcomes).to_csv('Output//{}_{}_no_converge_outcomes.csv'.format(course_name,root_guid[:4]))
    pd.DataFrame(less_than_3_questions).to_csv('Output//{}_{}_less_than_3_questions.csv'.format(course_name,root_guid[:4]))
    pd.DataFrame(normal_rasch_model).to_csv('Output//{}_{}_normal_rasch_model.csv'.format(course_name,root_guid[:4]))